In [2]:
import sys
sys.path.insert(0, r"E:\projects\effective_date_parser")

In [11]:
import os
import json
import pandas as pd
from pprint import pprint


import config

In [5]:
train_df = pd.read_csv(config.train_file)
pred_df = pd.read_csv(config.pred_file)

In [6]:
train_df.head()

,file,Start Date,Start Date-Answer
0,CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...,['This agreement shall begin upon the date of ...,NaN
1,EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...,"['July 11 , 2006']",07-11-2006
2,FulucaiProductionsLtd_20131223_10-Q_EX-10.9_83...,"['November 15, 2012']",11/15/2012
3,GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...,"['Feb 10, 2014']",02-10-2014
4,IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...,"['December 21, 2015']",12/21/2015


In [7]:
train_df.isna().sum()

file                   0
Start Date             0
Start Date-Answer    122
dtype: int64

In [8]:
train_df.fillna('')

,file,Start Date,Start Date-Answer
0,CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...,['This agreement shall begin upon the date of ...,
1,EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...,"['July 11 , 2006']",07-11-2006
2,FulucaiProductionsLtd_20131223_10-Q_EX-10.9_83...,"['November 15, 2012']",11/15/2012
3,GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...,"['Feb 10, 2014']",02-10-2014
4,IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...,"['December 21, 2015']",12/21/2015
...,...,...,...
404,NANOPHASETECHNOLOGIESCORP_11_01_2005-EX-99.1-D...,[],
405,"OPTIMIZEDTRANSPORTATIONMANAGEMENT,INC_07_26_20...",['EFFECTIVE DATE: The earlier of t...,
406,"VISIUMTECHNOLOGIES,INC_10_20_2004-EX-10.20-DIS...","['7th day of April, 2003']",04-07-2003
407,ENERGYXXILTD_05_08_2015-EX-10.13-Transportatio...,"['March 11, 2015']",03-11-2015


In [9]:
train_data = []

question_id = 1
question = "What is effective date?"
for index, row in train_df.iterrows():
    try :
        file_name = row["file"]
        name, ext = os.path.splitext(file_name)
        txt_file = os.path.join(config.contracts_dir, name+".txt")
        possible_ans = eval(row["Start Date"])
        actual_ans = row["Start Date-Answer"]

        if len(possible_ans) > 0:
            possible_ans = possible_ans[0]
        else:
            possible_ans = ""

        data_dict = dict()
        
        with open(txt_file, encoding="utf8") as f:
            data = f.read()

            data_dict["context"] = data

            qas_dict = dict()
            qas_dict["question"] = question
            qas_dict["id"] = question_id
            question_id += 1
            
            answer_dict = dict()
            if possible_ans != '' and actual_ans != '':
                if possible_ans in data:
                    answer_idx = data.find(possible_ans)
                    is_impossible = False
                    answer_dict = {
                        "text" : possible_ans,
                        "answer_start" : answer_idx
                    }
                else:
                    is_impossible = True
            else:
                is_impossible = True

            if is_impossible:
                qas_dict["is_impossible"] = is_impossible
                qas_dict["answers"] = []
            else:
                qas_dict["is_impossible"] = is_impossible
                qas_dict["answers"] = [answer_dict]

            data_dict["qas"] = [qas_dict]
        train_data.append(data_dict)
    except:
        pass

In [12]:
with open(config.processed_train_file,"w+") as f:
    json.dump(train_data,f)

<h2> Creating test data </h2>

In [13]:
test_data = []
question_id = 1
question = "What is effective date?"
file_idx = []
for index, row in pred_df.iterrows():
    try:
        file_name = row["file"]
        file_idx.append([file_name, question_id])
        name, ext = os.path.splitext(file_name)
        txt_file = os.path.join(config.contracts_dir, name+".txt")

        data_dict = dict()
        
        with open(txt_file, encoding="utf8") as f:
            data = f.read()

            data_dict["context"] = data

            qas_dict = dict()
            qas_dict["question"] = question
            qas_dict["id"] = question_id
            question_id += 1

            data_dict["qas"] = [qas_dict]

        test_data.append(data_dict)
    except:
        pass

In [15]:
with open(config.processed_pred_file,"w+") as f:
    json.dump(test_data,f)

In [17]:
pd.DataFrame(file_idx, columns=["file_name","id"]).to_csv(config.file_id_csv,index=False)